# 4. Combining yearly ERA5 datasets into one dataset

#### In this notebook:
- Testing various code used to engineer features in other notebooks

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Paths to my data directories
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

#### Read in latest turbine data
- Remove 4 turbines clearly outside of Germany given their coordinates in turbine dataset
- Remove two turbines at grid point latitude 55.57
    - index 3079 and 29763
- Remove two turbines at grid point longitude 5.72
    - index 26572, 30533
- Now only 751 unique grid points used

In [3]:
# Read in latest turbine data
# Now only 30,642 after removing 3 turbines outside bounding box
df_turbines = pd.read_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation_utc_2_2018_2023.pkl')
# df_turbines.info()

In [4]:
# Get unique grid points for masking the xarray dataset later
used_grid_points = df_turbines['nearest_grid_point'].unique()

In [5]:
used_grid_points.shape

(750,)

In [6]:
# Use these weights to calculate weighted means for my various weather params
grid_point_weights_dict = df_turbines['nearest_grid_point'].value_counts().to_dict()

In [7]:
len(grid_point_weights_dict.keys())

750

In [8]:
# df_turbines.drop([3079, 29763, 26572, 30533], inplace=True)
# df_turbines.reset_index(drop=True, inplace=True)
# df_turbines.to_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation_utc_2.pkl')

#### Read ERA5 datasets into Xarray

In [9]:
# Print files in my ERA5 dir
for filepath in data_path.joinpath('ERA5').iterdir():
    print(filepath)

/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined_subset_derived.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2020.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2021.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/.DS_Store
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined_subset_derived_dropped_masked.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined_subset_derived_dropped.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined_subset_2018_2023.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined_subset_derived_dropped_2018_2023.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2018.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_2019.nc
/Users/brand/my_code/meteoviz/course_project/data/ERA5/era5_combined_subset_derived_dropped_masked_2018_2023.nc
/

#### Note:
- Note: year 2023 has an unexpected dim called `expver`
    - "expver is used to tell the difference between the initial release (expver=5, called ERA5T) and validated ERA5 data (expver=1). In most cases, ERA5 is identical to ERA5T"
    - More here: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Dataupdatefrequency

In [122]:
ds_2018 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2018.nc'))

In [123]:
ds_2019 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2019.nc'))

In [124]:
ds_2020 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2020.nc'))

In [125]:
ds_2021 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2021.nc'))

In [126]:
ds_2022 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2022.nc'))

In [127]:
ds_2023 = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_2023.nc'))

#### Concat the xarray datasets along dimension `time`

In [128]:
datasets = [ds_2018, ds_2019, ds_2020, ds_2021, ds_2022, ds_2023]
ds_combined = xr.concat(datasets, dim='time')

#### Subset using unique used grid points (to reduce data volume and file size)

In [129]:
unique_lats = np.unique(df_turbines['nearest_grid_point'].apply(lambda coord: coord[0]))
unique_lons = np.unique(df_turbines['nearest_grid_point'].apply(lambda coord: coord[1]))
print(unique_lats.shape)
print(unique_lons.shape)

(30,)
(37,)


In [130]:
ds_combined_subset = ds_combined.sel(latitude=unique_lats, longitude=unique_lons)

#### Save combined, subsetted dataset

In [132]:
# ds_combined_subset.to_netcdf(data_path.joinpath('ERA5').joinpath('era5_combined_subset_2018_2023.nc'))

-----

----

## Read in combined, subsetted ERA5 dataset as `ds`

In [133]:
# Load back into xarray and check structure
ds = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_combined_subset_2018_2023.nc'))

In [134]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 37, latitude: 30, time: 48168)
Coordinates:
  * longitude  (longitude) float32 5.97 6.22 6.47 6.72 ... 14.47 14.72 14.97
  * latitude   (latitude) float32 47.57 47.82 48.07 48.32 ... 54.32 54.57 54.82
  * time       (time) datetime64[ns] 2018-01-01 ... 2023-06-30T23:00:00
Data variables:
    u100       (time, latitude, longitude) float32 ...
    v100       (time, latitude, longitude) float32 ...
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    i10fg      (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 11:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

-----

-----

### Check that all lats and lons match between data sets!
- All match!

In [135]:
# def check_dimensions_match(datasets, dim_name):
#     """Check if a given dimension's values are the same across multiple datasets."""
#     reference_values = datasets[0][dim_name].values
#     for ds in datasets[1:]:
#         if not (ds[dim_name].values == reference_values).all():
#             return False
#     return True

In [136]:
# Check latitude and longitude for all datasets
# latitude_match = check_dimensions_match(datasets, 'latitude')
# longitude_match = check_dimensions_match(datasets, 'longitude')

# if latitude_match and longitude_match:
#     print("All datasets have matching latitude and longitude values!")
# else:
#     if not latitude_match:
#         print("Datasets do not have matching latitude values.")
#     if not longitude_match:
#         print("Datasets do not have matching longitude values.")

------

## Calculate 10m and 100m mean wind speeds using vector components

In [137]:
ds.data_vars

Data variables:
    u100     (time, latitude, longitude) float32 ...
    v100     (time, latitude, longitude) float32 ...
    u10      (time, latitude, longitude) float32 ...
    v10      (time, latitude, longitude) float32 ...
    t2m      (time, latitude, longitude) float32 ...
    i10fg    (time, latitude, longitude) float32 ...
    msl      (time, latitude, longitude) float32 ...

In [138]:
def calc_wind_speed_using_ortho_components(u_zonal, v_meridional):
    """
    u_zondal (u10): wind component along local parallel of latitude; positive from west, negative from east
    v_meridional (v10): wind component along local meridian; positive from south, negative from north
    returns the magnitude of the wind vector (i.e. wind speed)
    """
    # use numpy.sqrt() as math.sqrt() only accepts scalar value not array
    mean_wind_speed = np.sqrt(u_zonal**2 + v_meridional**2)
    return mean_wind_speed

#### 10m mean wind speed

In [139]:
# Apply the function and assign to xarray dataset as a new data variable 
ds['mean_wind_speed_10m'] = xr.apply_ufunc(calc_wind_speed_using_ortho_components, ds['u10'], ds['v10'])

#### 100m mean wind speed

In [140]:
# Apply the function and assign to xarray dataset as a new data variable 
ds['mean_wind_speed_100m'] = xr.apply_ufunc(calc_wind_speed_using_ortho_components, ds['u100'], ds['v100'])

##### Get mean wind speed at specific hour and coordinate 

In [141]:
# Example used grid point: (47.82, 10.97)
ds.sel(time=['2022-01-01T00:00:00'], latitude=47.82, longitude=10.97)['mean_wind_speed_100m'].values

array([6.8206396], dtype=float32)

In [252]:
ds.data_vars

Data variables:
    t2m                        (time, latitude, longitude) float32 281.9 ... ...
    i10fg                      (time, latitude, longitude) float32 12.55 ... ...
    msl                        (time, latitude, longitude) float32 1.012e+05 ...
    mean_wind_speed_10m        (time, latitude, longitude) float32 6.006 ... ...
    mean_wind_speed_100m       (time, latitude, longitude) float32 10.22 ... ...
    wind_direction_angle_10m   (time, latitude, longitude) float32 232.2 ... ...
    wind_direction_angle_100m  (time, latitude, longitude) float32 236.2 ... ...

In [143]:
# ds['msl'].values

----

## Calculate wind direction angle in degrees using vector components

In [144]:
def calc_wind_direction_using_ortho_components(u_zonal, v_meridional):
    """
    u_zonal and v_meridional are the magnitudes of the component vectors
    use np.arctan2 method that can take scalar values as input
    """
    # Theta is direction in radians
    theta_radians = np.arctan2(-u_zonal, -v_meridional)
    
    # Convert radians to degrees
    theta_degrees = np.degrees(theta_radians)
    
    # Normalise to compass heading convention [0, 360) if value is negative
    theta_degrees = np.where(theta_degrees < 0, theta_degrees + 360, theta_degrees)
    
    return theta_degrees

#### 10m wind direction angle in degrees

In [145]:
# Apply the function and assign to xarray dataset as a new data variable 
ds['wind_direction_angle_10m'] = xr.apply_ufunc(calc_wind_direction_using_ortho_components, ds['u10'], ds['v10'])

#### 100m wind direction angle in degrees

In [146]:
# Apply the function and assign to xarray dataset as a new data variable 
ds['wind_direction_angle_100m'] = xr.apply_ufunc(calc_wind_direction_using_ortho_components, ds['u100'], ds['v100'])

In [147]:
ds.data_vars

Data variables:
    u100                       (time, latitude, longitude) float32 8.496 ... ...
    v100                       (time, latitude, longitude) float32 5.686 ... ...
    u10                        (time, latitude, longitude) float32 4.749 ... ...
    v10                        (time, latitude, longitude) float32 3.678 ... ...
    t2m                        (time, latitude, longitude) float32 ...
    i10fg                      (time, latitude, longitude) float32 ...
    msl                        (time, latitude, longitude) float32 ...
    mean_wind_speed_10m        (time, latitude, longitude) float32 6.006 ... ...
    mean_wind_speed_100m       (time, latitude, longitude) float32 10.22 ... ...
    wind_direction_angle_10m   (time, latitude, longitude) float32 232.2 ... ...
    wind_direction_angle_100m  (time, latitude, longitude) float32 236.2 ... ...

------

### Drop data vars that are no longer needed (and further reduce data volume)

In [148]:
ds = ds.drop(labels=['u100', 'v100', 'u10', 'v10'])

In [157]:
ds.data_vars

Data variables:
    t2m                        (time, latitude, longitude) float32 ...
    i10fg                      (time, latitude, longitude) float32 ...
    msl                        (time, latitude, longitude) float32 ...
    mean_wind_speed_10m        (time, latitude, longitude) float32 6.006 ... ...
    mean_wind_speed_100m       (time, latitude, longitude) float32 10.22 ... ...
    wind_direction_angle_10m   (time, latitude, longitude) float32 232.2 ... ...
    wind_direction_angle_100m  (time, latitude, longitude) float32 236.2 ... ...

#### Save latest Xarray dataset as NetCDF

In [150]:
# ds.to_netcdf(data_path.joinpath('ERA5').joinpath('era5_combined_subset_derived_dropped_2018_2023.nc'))

-----

### Filter `ds` using a mask with condition being the unique grid points connected to turbines!
- Create a boolean mask shape (30,37) using used grid points 
- Data vars will be nan at grid points NOT used in my analysis
- Keep only data var values at the 751 unique coordinate pairs used for the wind turbines; the rest set to nan

In [151]:
# a mask of False values (same truthiness as 0) with shape of lat and lon dimensions in ds (30,37)
bool_mask = np.zeros((len(ds['latitude']), len(ds['longitude'])), dtype=bool)

# Loop through unique coord pairs and get index where lat, lon match in xarray dataset
for lat, lon in used_grid_points:
    lat_idx = np.where(ds['latitude'] == lat)[0]
    lon_idx = np.where(ds['longitude'] == lon)[0]
    # Use broadcasting to insert True values into bool mask at True lat, lon indexes
    bool_mask[lat_idx, lon_idx] = True

# Create DataArray of the mask using same dims and coords as ds
da_mask = xr.DataArray(bool_mask, coords=[ds['latitude'], ds['longitude']], dims=['latitude', 'longitude'])

In [152]:
# Mask the dataset
ds_masked = ds.where(da_mask)

In [153]:
ds_masked

<xarray.Dataset>
Dimensions:                    (time: 48168, latitude: 30, longitude: 37)
Coordinates:
  * longitude                  (longitude) float32 5.97 6.22 ... 14.72 14.97
  * latitude                   (latitude) float32 47.57 47.82 ... 54.57 54.82
  * time                       (time) datetime64[ns] 2018-01-01 ... 2023-06-3...
Data variables:
    t2m                        (time, latitude, longitude) float32 nan ... nan
    i10fg                      (time, latitude, longitude) float32 nan ... nan
    msl                        (time, latitude, longitude) float32 nan ... nan
    mean_wind_speed_10m        (time, latitude, longitude) float32 nan ... nan
    mean_wind_speed_100m       (time, latitude, longitude) float32 nan ... nan
    wind_direction_angle_10m   (time, latitude, longitude) float32 nan ... nan
    wind_direction_angle_100m  (time, latitude, longitude) float32 nan ... nan
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 11:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

#### Save latest Xarray dataset as NetCDF

In [154]:
# ds_masked.to_netcdf(data_path.joinpath('ERA5').joinpath('era5_combined_subset_derived_dropped_masked_2018_2023.nc'))

------

In [155]:
ds['mean_wind_speed_10m'].mean(dim=['latitude', 'longitude']).to_dataframe()

,mean_wind_speed_10m
time,
2018-01-01 00:00:00,5.905850
2018-01-01 01:00:00,6.261999
2018-01-01 02:00:00,6.672714
2018-01-01 03:00:00,6.924845
2018-01-01 04:00:00,7.140243
...,...
2023-06-30 19:00:00,2.178707
2023-06-30 20:00:00,2.174161
2023-06-30 21:00:00,2.309697


In [317]:
ds_masked['mean_wind_speed_10m'].mean(dim=['latitude', 'longitude']).to_dataframe()

,mean_wind_speed_10m
time,
2018-01-01 00:00:00,5.978772
2018-01-01 01:00:00,6.309438
2018-01-01 02:00:00,6.685641
2018-01-01 03:00:00,6.894922
2018-01-01 04:00:00,7.060956
...,...
2023-06-30 19:00:00,1.842328
2023-06-30 20:00:00,1.792745
2023-06-30 21:00:00,1.922537


----

# Test deriving weighted means of weather features

#### Load in latest NetCDF file

In [48]:
# Load back into xarray and check structure
ds = xr.open_dataset(data_path.joinpath('ERA5').joinpath('era5_combined_subset_derived_dropped_masked_2018_2023.nc'))

In [49]:
ds.data_vars

Data variables:
    t2m                        (time, latitude, longitude) float32 ...
    i10fg                      (time, latitude, longitude) float32 ...
    msl                        (time, latitude, longitude) float32 ...
    mean_wind_speed_10m        (time, latitude, longitude) float32 ...
    mean_wind_speed_100m       (time, latitude, longitude) float32 ...
    wind_direction_angle_10m   (time, latitude, longitude) float32 ...
    wind_direction_angle_100m  (time, latitude, longitude) float32 ...

In [50]:
# Zeros array with same shape at lat and lon dimensions (30,37)
weights_array = np.zeros((len(ds['latitude']), len(ds['longitude'])))

# loop through all 750 grid points
for lat, lon in used_grid_points:
    lat_idx = np.where(ds['latitude'] == lat)[0][0]
    lon_idx = np.where(ds['longitude'] == lon)[0][0]
    # Use broadcasting to insert weights into weights array
    weights_array[lat_idx, lon_idx] = grid_point_weights_dict[(lat, lon)]
    
# create the DataArray of weights
da_weights = xr.DataArray(weights_array, coords=[ds['latitude'], ds['longitude']], dims=['latitude', 'longitude'])

# total of the weights
total_weight = da_weights.sum(dim=['latitude', 'longitude'])

# calculate the weighted means for all 7 of my data variables
weighted_mean_t2m = (ds['t2m'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight
weighted_mean_i10fg = (ds['i10fg'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight
weighted_mean_msl = (ds['msl'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight
weighted_mean_mean_wind_speed_10m = (ds['mean_wind_speed_10m'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight
weighted_mean_mean_wind_speed_100m = (ds['mean_wind_speed_100m'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight
weighted_mean_wind_direction_angle_10m = (ds['wind_direction_angle_10m'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight
weighted_mean_wind_direction_angle_100m = (ds['wind_direction_angle_100m'] * da_weights).sum(dim=['latitude', 'longitude']) / total_weight

In [52]:
weighted_mean_t2m.to_dataframe(name='weighted_t2m')

,weighted_t2m
time,
2018-01-01 00:00:00,282.357286
2018-01-01 01:00:00,282.071161
2018-01-01 02:00:00,281.867961
2018-01-01 03:00:00,281.645501
2018-01-01 04:00:00,281.230761
...,...
2023-06-30 19:00:00,291.207953
2023-06-30 20:00:00,289.938159
2023-06-30 21:00:00,289.225749


In [53]:
ds

<xarray.Dataset>
Dimensions:                    (time: 48168, latitude: 30, longitude: 37)
Coordinates:
  * longitude                  (longitude) float32 5.97 6.22 ... 14.72 14.97
  * latitude                   (latitude) float32 47.57 47.82 ... 54.57 54.82
  * time                       (time) datetime64[ns] 2018-01-01 ... 2023-06-3...
Data variables:
    t2m                        (time, latitude, longitude) float32 ...
    i10fg                      (time, latitude, longitude) float32 ...
    msl                        (time, latitude, longitude) float32 ...
    mean_wind_speed_10m        (time, latitude, longitude) float32 ...
    mean_wind_speed_100m       (time, latitude, longitude) float32 ...
    wind_direction_angle_10m   (time, latitude, longitude) float32 ...
    wind_direction_angle_100m  (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-09-11 11:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...